In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.tensorboard as tb

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

from torch import save
from torch import load
from os import path
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler

from torch import save
from torch import load
from os import path

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# set current directory

#%cd /content/drive/My Drive/Steering_Datasets_Rev1/CH2_002_Train/center
# !unzip center_only.zip

%cd /content/drive/My Drive/Colab Notebooks/CIS519 - Applied Machine Learning/Project

# DataLoader



In [0]:
class SteeringDataset(Dataset):
    def __init__(self, image_path, data_transforms=None):
        """
        Your code here
        Hint: Use the python csv library to parse labels.csv
        """
        # self.name_label_path = name_label_path
        self.image_path = image_path
        self.name = pd.read_csv(self.image_path + 'interpolated.csv', usecols=range(5,6))

        self.labels = pd.read_csv(self.image_path + 'interpolated.csv', usecols=range(6,7))
        self.center_data = pd.concat([self.name, self.labels], axis=1) #combine image name and label dataframes
        self.center_data = self.center_data[self.center_data["filename"].str.contains('center')] # only keep center image names and labels

        self.name = pd.DataFrame(self.center_data[self.center_data.columns[0]]) # center images names
        self.len = self.name.shape[0]
        self.labels = pd.DataFrame(self.center_data[self.center_data.columns[1]]) # center image labels

        self.transform = transforms.Compose([
                                transforms.ToTensor()])  
                                #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        # bin angles into 100 classes

        # classes = 100
        # bins = np.linspace(-2.1,2,classes)
        # labels = np.linspace(1,classes-1,classes-1).astype(int)
        # self.labels = pd.cut(self.labels['angle'], bins = bins, labels = labels)

        print(self.labels)

    def __len__(self):
        """
        Your code here
        """
        return self.len

    def __getitem__(self, idx):
        """
        Your code here
        return a tuple: img, label
        """
        # img = Image.open(str(self.name.iloc[idx, 0]))
        img = Image.open(str(self.name.iloc[idx, 0])[7:]) # [7:] removes 'center/' before the image number since the zipped images just have numbers
        img = img.crop((0, 240, 640, 480))  # crop image (remove above horizon). Original dimensions: 640,480. changing to 640,240 
        img = img.resize((320,240)) #resize image
        #img = img.resize((128,128)) #resize image

        img = self.transform(img)
        
        label = self.labels.iloc[idx][0]
        return img, label


# Save and Load Models

In [0]:
def load_data(dataset_path, data_transforms, num_workers, batch_size, shuffle):
    dataset = SteeringDataset(dataset_path, data_transforms)
    return DataLoader(dataset, num_workers=num_workers, batch_size=batch_size, shuffle=shuffle)

def train_test_split(args, dataset):
    # Indices of Split
    size = len(dataset)
    indices = list(range(size))
    train_size = int(args.test_fraction * size)
    train_indices, test_indices = indices[train_size:], indices[:train_size]

    # SubsetRandomSamplers
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(test_indices)

    # DataLoaders
    train = DataLoader(dataset, batch_size=args.train_batch_size, sampler=train_sampler)
    test = DataLoader(dataset, batch_size=args.test_batch_size, sampler=valid_sampler)

    return train, test

def save_model(model, file_name):
    #if isinstance(model, CNNClassifier):
    return save(model.state_dict(), path.join(path.abspath(''), file_name))

    #raise ValueError("model type '%s' not supported!"%str(type(model)))

def load_model(file_name):
    r = CustomModel()
    r.load_state_dict(load(path.join(path.abspath(''), file_name), map_location='cpu'))
    return r

# Loss

In [0]:
class ClassificationLoss(torch.nn.Module):
    def forward(self, input, target):
        """
        Your code here
        Compute mean(-log(softmax(input)_label))
        @input:  torch.Tensor((B,C)), where B = batch size, C = number of classes
        @target: torch.Tensor((B,), dtype=torch.int64)
        @return:  torch.Tensor((,))
        Hint: Don't be too fancy, this is a one-liner
        """
        m = nn.MSELoss()
        return torch.sqrt(m(input.view(input.size(0)),target))

# Model

In [0]:
class CNNClassifier(torch.nn.Module):
    def __init__(self):
        """
        Your code here
        """
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=3)
        self.conv2 = nn.Conv2d(5, 15, kernel_size=3)
        self.fc1 = nn.Linear(67860, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50,1)


    def forward(self, x):
        """
        Your code here
        @x: torch.Tensor((B,3,64,64))
        @return: torch.Tensor((B,6))
        """
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x



def vgg_model():
    model = models.vgg11(pretrained=False)
    for param in model.parameters():
        param.requires_grad = True  # False: Freezes the weights of the pre-trained model
    #model.classifier[3] = nn.Linear(4096, 4096)
    model.classifier[6] = nn.Linear(4096, 1) # replace last fc layer

    return model



def resnet():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False  # False: Freezes the weights of the pre-trained model
    #model.fc = nn.Sequential(nn.Linear(2048, 2048),
                             #nn.ReLU(),
                             #nn.Linear(2048, 1))
    model.fc = nn.Linear(2048, 1)

    return model

# Parameters

In [0]:
# Parameters

class Args(object):
    def __init__(self, model):

        # load_data
        self.data_path = '/content/drive/My Drive/Colab Notebooks/CIS519 - Applied Machine Learning/Project/' #'/content/drive/My Drive/Steering_Datasets_Rev1/CH2_002_Train/'
        self.transforms = None  # not in use
        self.num_workers = 0  # not in use
        self.train_batch_size = 64
        self.test_batch_size = 1  # b/c batches aren't needed for testing
        self.shuffle = False  # not in use

        #train_test_split
        self.test_fraction = 0.3

        # Device
        # self.device = torch.device("cuda")

        # train
        self.learning_rate = 0.001 
        self.momentum = 0.9 # not in use 
        self.epochs = 8
        self.optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate)
        self.criterion = ClassificationLoss() #nn.CrossEntropyLoss() #RMSELoss().   CrossEntropyLoss: lable.long() conversion needed in train()

        # test
        self.correct_threshold = 0.1  # compared with prediction loss


# Train

In [0]:
def train(args,model,train_data, test_data):
    """
    Your code here
    """
    
    # initialize weights

    for m in model.modules():
        if isinstance(m, torch.nn.Linear):
            torch.nn.init.normal_(m.weight, mean=0, std=0.1)
            torch.nn.init.constant_(m.bias, 0.1)
        if isinstance(m, torch.nn.Conv2d):
            torch.nn.init.normal_(m.weight, mean=0, std=0.1)


    # Model
    model.train()
    # model.to(args.device)
    
    # Optimizer & Criterion
    optimizer = args.optimizer
    criterion = args.criterion
    # criterion.to(args.device)


    # Book Keeping
    train_losses = []
    train_counter = []
    num_steps_per_epoch = len([i for i in range(len(train_data)) if i%10==0])


    # Epoch Loop
    for epoch in range(1, args.epochs+1):

        stepper = 0
        model.train()
        # Batch Loop
        for batch_idx, (data, label) in enumerate(train_data):

            # data, label = data.to(args.device), label.to(args.device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)

            loss.backward()
            optimizer.step()

            # Printing & Logging
            if batch_idx % 1 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, (batch_idx+1) * len(data), len(train_data)*len(data),
                  100. * batch_idx / len(train_data), loss.item()))                
                train_losses.append(loss.item())
                train_counter.append((batch_idx*64) + ((epoch-1)*len(train_data)))
                stepper += 1

            if batch_idx % 4 == 0:
                print('Testing...')
                valid_loss = test(args, model, test_data)
                print("validation loss = ", valid_loss)

        # check validation loss every epoch
        
        valid_loss = test(args, model, test_data)
        print("validation loss = ", valid_loss)

    #return model
    save_model(model,'cnn_zach')

In [0]:
def test(args, model, test_data):
path
    # Model
    model.eval()
    # model = model.to(args.device)

    # Loss
    criterion = args.criterion
    # criterion.to(args.device)

    # Book Keeping
    test_loss = 0
    correct = 0  # Since no classes: correct++ occurs when prediction is sufficiently close to target (within a threshold)
    test_progress = 0

    # Data Loop
    with torch.no_grad():
        for data, target in test_data:

            # Use Model
            # data, target = data.to(args.device), target.to(args.device)
            output = model(data)
            loss = criterion(output, target).item()
            test_loss += loss

            # #pred = output.argmax(dim=1, keepdim=True)  # don't need since no classes.
            # if loss < args.correct_threshold:
            #     correct += 1 #pred.eq(target.view_as(pred)).sum().item()

            # ## TODO: Build List of predictions, targets, and loss for each image so that data can be overlayed onto the images for the output video

            # Print Progress
            test_progress += 1
            if test_progress % 100 == 0:
              print('Images Processed: ', test_progress, ', Overall Progress: ', int(100*test_progress/len(test_data)*args.test_batch_size), '%')

        test_loss /= len(test_data)

        # print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.
        #       format(test_loss, correct, len(test_data.dataset), 100. * correct / len(test_data.dataset)))  
    return test_loss

In [0]:
#model = CNNClassifier()
#model = vgg_model()    # predominantly pretrained deep nn. untrained output layer (4096->1)
model = resnet()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
# Parameters
args = Args(model)

# Load Data & Train/Test Split
print('Loading Data...')
train_data, test_data = train_test_split(args, SteeringDataset(args.data_path, args.transforms))

print('Training...')
train(args, model, train_data, test_data)



Loading Data...
           angle
2       0.001039
5       0.003491
8       0.003491
11      0.005236
14      0.006234
...          ...
101382  0.000000
101385  0.000000
101388  0.000000
101391  0.000000
101394  0.000000

[33808 rows x 1 columns]
Training...
Train Epoch: 1 [64/23680 (0%)]	Loss: 1.743885


In [0]:
# Test Model
#test(args, model, test_data) 